# Give Me Some Credit

This competition requires participants to improve on the state of the art in credit scoring, by predicting the probability that somebody will experience financial distress in the next two years.

The goal of this competition is to build a model that borrowers can use to help make the best financial decisions.

A quick desk research on Credit Scoring provided the following information:

Biggest 5 factors that affects credit scoring:
1. Payment History – 35% of credit score
2. Amounts Owed – 30% of credit score
3. Length of Credit History – 15% of credit score
4. New Credit – 10% of credit score
5. Types of Credit In Use – 10% of credit score

In [1]:
# Import packages
import pandas as pd

# Display options
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows',200)
pd.set_option('display.max_colwidth',200)

In [2]:
# Understanding Data Dictionary
xls = pd.ExcelFile('Data Dictionary.xls')
xls.sheet_names
data_dictionary = xls.parse(xls.sheet_names[0])
data_dictionary 

,Variable Name,Description,Type
0,SeriousDlqin2yrs,Person experienced 90 days past due delinquency or worse,Y/N
1,RevolvingUtilizationOfUnsecuredLines,Total balance on credit cards and personal lines of credit except real estate and no installment debt like car loans divided by the sum of credit limits,percentage
2,age,Age of borrower in years,integer
3,NumberOfTime30-59DaysPastDueNotWorse,Number of times borrower has been 30-59 days past due but no worse in the last 2 years.,integer
4,DebtRatio,"Monthly debt payments, alimony,living costs divided by monthy gross income",percentage
5,MonthlyIncome,Monthly income,real
6,NumberOfOpenCreditLinesAndLoans,Number of Open loans (installment like car loan or mortgage) and Lines of credit (e.g. credit cards),integer
7,NumberOfTimes90DaysLate,Number of times borrower has been 90 days or more past due.,integer
8,NumberRealEstateLoansOrLines,Number of mortgage and real estate loans including home equity lines of credit,integer
9,NumberOfTime60-89DaysPastDueNotWorse,Number of times borrower has been 60-89 days past due but no worse in the last 2 years.,integer


In [3]:
#First look at training data
training = pd.read_csv('cs-training.csv')
training.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
#Examine values
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 12 columns):
Unnamed: 0                              150000 non-null int64
SeriousDlqin2yrs                        150000 non-null int64
RevolvingUtilizationOfUnsecuredLines    150000 non-null float64
age                                     150000 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    150000 non-null int64
DebtRatio                               150000 non-null float64
MonthlyIncome                           120269 non-null float64
NumberOfOpenCreditLinesAndLoans         150000 non-null int64
NumberOfTimes90DaysLate                 150000 non-null int64
NumberRealEstateLoansOrLines            150000 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    150000 non-null int64
NumberOfDependents                      146076 non-null float64
dtypes: float64(4), int64(8)
memory usage: 13.7 MB


In [5]:
training.drop(training.columns[0],axis=1,inplace=True)

In [6]:
training.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,1.202690e+05,150000.000000,150000.000000,150000.000000,150000.000000,146076.000000
mean,0.066840,6.048438,52.295207,0.421033,353.005076,6.670221e+03,8.452760,0.265973,1.018240,0.240387,0.757222
std,0.249746,249.755371,14.771866,4.192781,2037.818523,1.438467e+04,5.145951,4.169304,1.129771,4.155179,1.115086
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.029867,41.000000,0.000000,0.175074,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.154181,52.000000,0.000000,0.366508,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.559046,63.000000,0.000000,0.868254,8.249000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,50708.000000,109.000000,98.000000,329664.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,20.000000


# Data Cleaning

Let's shorten the column names.

In [7]:
#Rename columns
new_col = ['dlq','util_ratio','age','ph_30to59','debtratio','monthlyincome',\
           'open_credit_loans','ph_90','realestate_lines','ph_60to89','dependents']
rename_col = {col:new_col[i] for i, col in enumerate(training.columns)}

training.rename(rename_col,axis=1,inplace=True)

In [8]:
#Data Dictionary
data_dictionary['New Column Names'] = new_col
data_dictionary

,Variable Name,Description,Type,New Column Names
0,SeriousDlqin2yrs,Person experienced 90 days past due delinquency or worse,Y/N,dlq
1,RevolvingUtilizationOfUnsecuredLines,Total balance on credit cards and personal lines of credit except real estate and no installment debt like car loans divided by the sum of credit limits,percentage,util_ratio
2,age,Age of borrower in years,integer,age
3,NumberOfTime30-59DaysPastDueNotWorse,Number of times borrower has been 30-59 days past due but no worse in the last 2 years.,integer,ph_30to59
4,DebtRatio,"Monthly debt payments, alimony,living costs divided by monthy gross income",percentage,debtratio
5,MonthlyIncome,Monthly income,real,monthlyincome
6,NumberOfOpenCreditLinesAndLoans,Number of Open loans (installment like car loan or mortgage) and Lines of credit (e.g. credit cards),integer,open_credit_loans
7,NumberOfTimes90DaysLate,Number of times borrower has been 90 days or more past due.,integer,ph_90
8,NumberRealEstateLoansOrLines,Number of mortgage and real estate loans including home equity lines of credit,integer,realestate_lines
9,NumberOfTime60-89DaysPastDueNotWorse,Number of times borrower has been 60-89 days past due but no worse in the last 2 years.,integer,ph_60to89


In [9]:
training.isnull().sum() #There are null values under monthly income column and dependents columns

dlq                      0
util_ratio               0
age                      0
ph_30to59                0
debtratio                0
monthlyincome        29731
open_credit_loans        0
ph_90                    0
realestate_lines         0
ph_60to89                0
dependents            3924
dtype: int64

Since the total number of null values are not substantial (<10%) and monthly income and number of dependents are not within the top 5 factors of credit scoring, we will use a simple method to impute the nan values.

In [10]:
#Impute with median values
training.monthlyincome.fillna(training.monthlyincome.median(),inplace=True)
training.dependents.fillna(training.dependents.median(),inplace=True)

In [11]:
training.isnull().sum()  #Double check

dlq                  0
util_ratio           0
age                  0
ph_30to59            0
debtratio            0
monthlyincome        0
open_credit_loans    0
ph_90                0
realestate_lines     0
ph_60to89            0
dependents           0
dtype: int64

In [12]:
#First look at our test data
test = pd.read_csv('cs-test.csv')
test.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,NaN,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0
1,2,NaN,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0
2,3,NaN,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0
3,4,NaN,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0
4,5,NaN,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0


In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101503 entries, 0 to 101502
Data columns (total 12 columns):
Unnamed: 0                              101503 non-null int64
SeriousDlqin2yrs                        0 non-null float64
RevolvingUtilizationOfUnsecuredLines    101503 non-null float64
age                                     101503 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    101503 non-null int64
DebtRatio                               101503 non-null float64
MonthlyIncome                           81400 non-null float64
NumberOfOpenCreditLinesAndLoans         101503 non-null int64
NumberOfTimes90DaysLate                 101503 non-null int64
NumberRealEstateLoansOrLines            101503 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    101503 non-null int64
NumberOfDependents                      98877 non-null float64
dtypes: float64(5), int64(7)
memory usage: 9.3 MB


In [14]:
test.drop(test.columns[0:2],axis=1,inplace=True)

In [15]:
#Rename columns
test.rename(rename_col,axis=1,inplace=True)

In [16]:
#Check for null values
test.isnull().sum()

util_ratio               0
age                      0
ph_30to59                0
debtratio                0
monthlyincome        20103
open_credit_loans        0
ph_90                    0
realestate_lines         0
ph_60to89                0
dependents            2626
dtype: int64

In [17]:
#Impute with median values from training set
test.monthlyincome.fillna(training.monthlyincome.median(),inplace=True)
test.dependents.fillna(training.dependents.median(),inplace=True)

In [18]:
test.isnull().sum() #Double check

util_ratio           0
age                  0
ph_30to59            0
debtratio            0
monthlyincome        0
open_credit_loans    0
ph_90                0
realestate_lines     0
ph_60to89            0
dependents           0
dtype: int64

In [19]:
# training.to_pickle('datasets/clean_training.pickle')
# test.to_pickle('datasets/clean_test.pickle')
# data_dictionary.to_pickle('datasets/data_dict.pickle')